# Aufgabe 7: AutoML

AutoML stellt Methoden und Prozesse zur Verfügung, um maschinelles Lernen auch Nicht-Machine-Learning-Experten zugänglich zu machen. Die Universität Freiburg hat eine Internetseite mit weiterführenden Informationen und einigen AutoML-Kackages zusammengestellt, die unter folgenden Link zu erreichen ist: https://www.ml4aad.org/automl/.

###### (a) Wählen Sie ein AutoML Package. Begründen Sie Ihre Auswahl.

Wir haben uns für _H2O AutoML_ entschieden, weil es in Hinblick auf die Reproduzierbarkeit der Ergebnisse am leichtesten auf verschiedenen Plattformen installierbar ist.

Bei _auto-sklearn_ beispielsweise fehlten die Berechtigungen, um die benötigten Dependencies auf SDIL zu installieren.
Für andere AutoML-Bibliotheken wurde lediglich ein git-repo zur Verfügung gestellt, weshalb diese nicht einfach über eine `requirements.txt` installiert werden können.

Zudem fanden wir die Dokumentation zu _H2O AutoML_ auf den ersten Blick sehr übersichtlich.

###### (b) Führen Sie die Klassifikationsaufgabe von Aufgabe 3 mit AutoML durch. Vergleichen Sie die Ergebnisse mit den Ergebnissen aus Aufgabe 3.

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

In [1]:
# Import Libraries
import time
import multiprocessing

import pandas as pd
import numpy as np

from sklearn import svm
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

import h2o
from h2o.automl import H2OAutoML
from h2o.frame import H2OFrame

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [2]:
# initialize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,14 mins 44 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.2
H2O cluster version age:,4 months and 6 days !!!
H2O cluster name:,H2O_from_python_carmen_v6ncs2
H2O cluster total nodes:,1
H2O cluster free memory:,3.868 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [3]:
# Load dataset iris
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
iris = pd.read_csv(url, names=names)

In [ ]:
# Split-out validation dataset
array = iris.values
X = array[:,0:4]
Y = array[:,4]
test_size = 0.20
seed = 7
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

In [ ]:
# prepare data by creating a train and test array and transforming it into an h2o frame
train_array = np.column_stack([X_train, y_train])
test_array = np.column_stack([X_test, y_test])

train = H2OFrame(python_obj=train_array, column_names=names)
test = H2OFrame(python_obj=test_array, column_names=names)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
# Identify predictors and response
x = names
y = "class"
x.remove(y)

In [ ]:
# set number of models to 20 and start training
aml = H2OAutoML(max_models=20, seed=1, nfolds=10)
aml.train(x=x, y=y, training_frame=train)                                 

AutoML progress: |███████████████████

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
display(lb.head(rows=lb.nrows)) # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
display(aml.leader)

In [ ]:
# predict on test data
predictions = aml.leader.predict(test).as_data_frame()['predict'].tolist()

In [ ]:
# get target for test data
y_test = test.as_data_frame()['class'].tolist()

In [ ]:
# print results
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

Der DeepLearning-Ansatz wird von AutoML als der geeignetste betrachtet. Dieser erreicht auf den Trainingsdaten eine Accuracy von 0.8667 (Deep Learning-Ansätze sind hier leider nicht reproduzierbar: "H2O Deep Learning models are not reproducible by default for performance reasons", http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html#automl-interface)und liegt damit unterhalb der Accuracy des Models mit der höchsten Accuracy aus Aufgabe drei (0.967). 

# Zweiter Datensatz

In [ ]:
# Load dataset
url_heart = "./data/heart.csv"
names_heart =  ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 
                 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved', 'exercise_induced_angina',
                 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']
heart = pd.read_csv(url_heart, names=names_heart)
heart.head()

In [ ]:
# drop first row
heart = heart.drop(heart.index[0])
heart.head()

In [ ]:
# Split-out validation dataset
array_heart = heart.values
X_heart = array_heart[:,0:13]
Y_heart = array_heart[:,13]
test_size_heart = 0.20
seed_heart = 10
X_heart_train, X_heart_test, y_heart_train, y_heart_test = model_selection.train_test_split(X_heart, Y_heart, test_size=test_size_heart, random_state=seed_heart)

In [ ]:
# prepare data by creating a train and test array and transforming it into an h2o frame
train_heart_array = np.column_stack([X_heart_train, y_heart_train])
test_heart_array = np.column_stack([X_heart_test, y_heart_test])

train_heart = H2OFrame(python_obj=train_heart_array, column_names=names_heart)
test_heart = H2OFrame(python_obj=test_heart_array, column_names=names_heart)

In [ ]:
# Identify predictors and response
x_heart = names_heart
y_heart = "target"
x_heart.remove(y_heart)

In [ ]:
# set number of models to 20 and start training
aml_heart = H2OAutoML(max_models=20, seed=1, nfolds=10)
aml_heart.train(x=x_heart, y=y_heart, training_frame=train_heart)                                 

In [ ]:
# View the AutoML Leaderboard
lb_heart = aml_heart.leaderboard
display(lb_heart.head(rows=lb_heart.nrows)) # Print all rows instead of default (10 rows)

In [ ]:
# The leader model is stored here
display(aml_heart.leader)

In [ ]:
# predict on test data
predictions_heart = aml_heart.leader.predict(test_heart).as_data_frame()['predict'].tolist()

In [ ]:
# get target for test data
y_heart_test = test_heart.as_data_frame()['target'].tolist()

# predictions are continuous --> set threshold for final prediction to 0.5
predictions_heart = pd.DataFrame(predictions_heart)
mask = predictions_heart>0.5
predictions_heart_binary = mask.astype(int)

In [ ]:
# for comparability with ex 3
def sensitivity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    return sensitivity

def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return specificity

In [ ]:
# print results
print('accuracy: '+ str(accuracy_score(y_heart_test, predictions_heart_binary)))
print('sensitivity:' + str(sensitivity_score(y_heart_test, predictions_heart_binary)))
print('specificity:' + str(specificity_score(y_heart_test, predictions_heart_binary)))

#print(confusion_matrix(y_heart_test, predictions_heart_binary))
#print(classification_report(y_heart_test, predictions_heart_binary))

Auch hier ist die Accuracy mit ca. 0.7377 um etwa 10 Prozentpunkte schlechter als in Aufgabe 3.

###### (c) Was ist Ihre Meinung zu AutoML?

- AutoML nimmt einem viel Arbeit ab, was gerade für Anfänger hilfreich ist. Allerdings verliert man hierdurch auch etwas Kontrolle. Die Ergebnisse sind scheinbar schlechter als in Aufgabe 3. Jedoch ist es schwer nachzuvollziehen wieso, da eben alles automatisch passiert. Zudem ist nicht klar, wieso welche Modelle getestet wurden. 

- "AutoML can only guarantee reproducibility under certain conditions. H2O Deep Learning models are not reproducible by default for performance reasons", dies ist natürlich sehr ungünstig.http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html#automl-interface, die eingeschränkte Reproduzierbarkeit ist ein Problem.

- Das Training dauert relativ lange, weshalb es etwas enttäuschend ist, die Ergebnisse aus Aufgabe 3 nicht toppen zu können. 


- "goal of finding the “best” model without any prior knowledge", https://medium.com/analytics-vidhya/gentle-introduction-to-automl-from-h2o-ai-a42b393b4ba2 --> diese Erfahrung können wir nicht teilen. Man kann mit wenig Ahnung ein akzeptables Ergebnis erreichen, jedoch waren unsere Ergebnisse nicht im sehr guten Bereich.


- "The current version of AutoML trains and cross-validates the following algorithms (in the following order): three pre-specified XGBoost GBM (Gradient Boosting Machine) models, a fixed grid of GLMs, a default Random Forest (DRF), five pre-specified H2O GBMs, a near-default Deep Neural Net, an Extremely Randomized Forest (XRT), a random grid of XGBoost GBMs, a random grid of H2O GBMs, and a random grid of Deep Neural Nets. In some cases, there will not be enough time to complete all the algorithms, so some may be missing from teh leaderboard. AutoML then trains two Stacked Ensemble models." http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html --> Unseres Erachtens nach sollten noch mehr verschiedene Modelle ausprobiert werden. 